In [1]:
# Check P2M, M2P and M2M
import numpy as np
from pcm_fmm import pcm_fmm
p = 10
ngrid = 6
vscales, w, grid, vgrid = pcm_fmm.init_globals(p, p, ngrid)
vscalesm = vscales[:(p+1)*(p+1)]
src = np.array([0.5,0.4,0.3], dtype=np.float64)
dst = np.array([1, 1.1, 2.2], dtype=np.float64)
p2p = np.linalg.norm(src-dst)**-1
sph1_c = np.zeros(3)
sph1_r = 1
sph1_coef = pcm_fmm.fmm_p2m(src-sph1_c, sph1_r, p, vscalesm)
m2p = pcm_fmm.fmm_m2p(dst-sph1_c, sph1_r, p, vscalesm, sph1_coef)
print("|| P2M + M2P - P2P ||  /  || P2P || =", abs((m2p-p2p)/p2p))
sph2_c = -np.ones(3)
sph2_r = sph1_r + np.linalg.norm(sph2_c-sph1_c)
sph2_coef = pcm_fmm.fmm_p2m(src-sph2_c, sph2_r, p, vscalesm)
sph2_coef_m2m = np.zeros_like(sph2_coef)
pcm_fmm.fmm_m2m_baseline(sph1_c-sph2_c, sph1_r, sph2_r, p, vscalesm, sph1_coef, sph2_coef_m2m)
print("|| P2M(1) + M(1)2M(2) - P2M(2) ||  /  || P2M(2) || =",
      np.linalg.norm(sph2_coef_m2m-sph2_coef) / np.linalg.norm(sph2_coef))
sph3_c = np.array([0, 0, -1])
sph3_r = sph1_r + np.linalg.norm(sph3_c-sph1_c)
sph3_coef = pcm_fmm.fmm_p2m(src-sph3_c, sph3_r, p, vscalesm)
sph3_coef_m2m = np.zeros_like(sph3_coef)
pcm_fmm.fmm_m2m_ztranslate((sph1_c-sph3_c)[2], sph1_r, sph3_r, p, vscalesm, sph1_coef, sph3_coef_m2m)
print("|| P2M(1) + M(1)TZ2M(3) - P2M(3) ||  /  || P2M(3) || =",
      np.linalg.norm(sph3_coef_m2m-sph3_coef) / np.linalg.norm(sph3_coef))

|| P2M + M2P - P2P ||  /  || P2P || = 1.2811025058542572e-07
|| P2M(1) + M(1)2M(2) - P2M(2) ||  /  || P2M(2) || = 3.244277800823038e-16
|| P2M(1) + M(1)TZ2M(3) - P2M(3) ||  /  || P2M(3) || = 1.8747588097912796e-16


In [2]:
# Check M2L, L2P and L2L
import numpy as np
from pcm_fmm import pcm_fmm
pl = 10
pm = 10
ngrid = 6
vscales, w, grid, vgrid = pcm_fmm.init_globals(pm, pl, ngrid)
src = np.array([0.5,0.4,0.3], dtype=np.float64)
dst = np.array([1, 1.1, 2.2], dtype=np.float64)
p2p = np.linalg.norm(src-dst)**-1
# Compute multipole coefficients
sph1_c = np.zeros(3)
sph1_r = 2
sph1_coef = pcm_fmm.fmm_p2m(src-sph1_c, sph1_r, pm, vscales[:(pm+1)**2])
# Compute local expansion
sph2_c = np.array([1, 1, 2], dtype=np.float64)
sph2_r = 1.3
sph2_coef = np.zeros((pl+1)*(pl+1))
pcm_fmm.fmm_m2l_baseline(sph1_c-sph2_c, sph1_r, sph2_r, pm, pl, vscales, sph1_coef, sph2_coef)
l2p = pcm_fmm.fmm_l2p(dst-sph2_c, sph2_r, pl, vscales[:(pl+1)**2], sph2_coef)
print("|| P2M + M2L + L2P - P2P ||  /  || P2P || =", abs((l2p-p2p)/p2p))
sph3_c = np.array([1, 1, 2], dtype=np.float64)
sph3_r = 0.6
sph3_coef = np.zeros_like(sph2_coef)
pcm_fmm.fmm_l2l_baseline(sph2_c-sph3_c, sph2_r, sph3_r, pl, vscales[:(pl+1)**2], sph2_coef, sph3_coef)
l2l2p = pcm_fmm.fmm_l2p(dst-sph3_c, sph3_r, pl, vscales[:(pl+1)**2], sph3_coef)
print("|| L(1)2L(2) + L(2)2P - L(1)2P ||  /  || L(1)2P || =", abs((l2l2p-l2p)/l2p))
sph3_c = np.array([1.1, 1.1, 2.1], dtype=np.float64)
sph3_r = 0.6
sph3_coef = np.zeros_like(sph2_coef)
pcm_fmm.fmm_l2l_baseline(sph2_c-sph3_c, sph2_r, sph3_r, pl, vscales[:(pl+1)**2], sph2_coef, sph3_coef)
l2l2p = pcm_fmm.fmm_l2p(dst-sph3_c, sph3_r, pl, vscales[:(pl+1)**2], sph3_coef)
print("|| L(1)2L(3) + L(3)2P - L(1)2P ||  /  || L(1)2P || =", abs((l2l2p-l2p)/l2p))

|| P2M + M2L + L2P - P2P ||  /  || P2P || = 1.2813589395338697e-07
|| L(1)2L(2) + L(2)2P - L(1)2P ||  /  || L(1)2P || = 0.0
|| L(1)2L(3) + L(3)2P - L(1)2P ||  /  || L(1)2P || = 6.946662007989605e-16


In [2]:
# Check rotation-based M2M
import numpy as np
from pcm_fmm import pcm_fmm
p = 10
ngrid = 6
vscales, w, grid, vgrid = pcm_fmm.init_globals(p, p, ngrid)
vscalesm = vscales[:(p+1)*(p+1)]
sph1_c = np.zeros(3)
sph1_r = 1
sph1_coef = np.ones_like(vscalesm)#np.random.rand((p+1)*(p+1))
sph2_c = np.array([1, 10, 2], dtype=np.float64)
sph2_r = sph1_r + np.linalg.norm(sph2_c-sph1_c)
#sph2_r = 1
sph2_coef_baseline = np.zeros_like(sph1_coef)
sph2_coef_rotation = np.zeros_like(sph1_coef)
pcm_fmm.fmm_m2m_baseline(sph1_c-sph2_c, sph1_r, sph2_r, p, vscalesm, sph1_coef, sph2_coef_baseline)
tmp1 = sph1_c - sph2_c
r = np.linalg.norm(tmp1)
tmp1 = tmp1 / r
nsign = -1 if tmp1[2] >= 0 else 1
tmp2 = np.array([tmp1[1], tmp1[2]-nsign, tmp1[0]])
tmp2 = tmp2 / np.linalg.norm(tmp2)
tmp3 = np.eye(3) - 2*tmp2.reshape(-1, 1)@tmp2.reshape(1,-1)
tmp4 = pcm_fmm.fmm_sph_rotate(p, tmp3, sph1_coef)
pcm_fmm.fmm_m2m_ztranslate(nsign*r, sph1_r, sph2_r, p, vscalesm, tmp4, sph2_coef_rotation)
sph2_coef_rotation2 = pcm_fmm.fmm_sph_rotate(p, tmp3, sph2_coef_rotation)
tmp5 = sph2_coef_rotation2 - sph2_coef_baseline
print(np.linalg.norm(tmp5) / np.linalg.norm(sph2_coef_baseline))
#print(sph2_coef_rotation2)
#print(sph2_coef_baseline)
sph2_coef_reflection = np.zeros_like(sph2_coef_rotation2)
pcm_fmm.fmm_m2m_reflection(sph1_c-sph2_c, sph1_r, sph2_r, p, vscalesm, sph1_coef, sph2_coef_reflection)
#print(sph2_coef_reflection)
#print(sph2_coef_baseline)
tmp6 = sph2_coef_reflection - sph2_coef_baseline
print(np.linalg.norm(tmp6) / np.linalg.norm(sph2_coef_baseline))
%timeit pcm_fmm.fmm_m2m_baseline(sph1_c-sph2_c, sph1_r, sph2_r, p, vscalesm, sph1_coef, sph2_coef_baseline)
%timeit pcm_fmm.fmm_m2m_reflection(sph1_c-sph2_c, sph1_r, sph2_r, p, vscalesm, sph1_coef, sph2_coef_reflection)

4.8213164346353355e-16
5.1464408489691005e-16
19 µs ± 278 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
17.3 µs ± 110 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [3]:
0.5*(-171.72339167+170.58806384)

-0.5676639150000113

In [8]:
import numpy as np
from time import time
from pcm_fmm import pcm_fmm
from mydx import mydx
mydx.init()
nbasis, ngrid, nsph = mydx.get_sizes()
csph, rsph = mydx.get_spheres(nsph)
ui = mydx.get_ui(nsph, ngrid)
p = int(nbasis**0.5)-1
print("nsph = {}\np = {}\nngrid = {}".format(nsph, p, ngrid))
vscales, w, grid, vgrid = pcm_fmm.init_globals(p, p, ngrid)
ind = np.arange(1, nsph+1, dtype=np.int32)
cluster, children, parent, cnode, rnode, snode = pcm_fmm.tree_init(csph, rsph, ind)
#print(ind)
#print(cluster)
#print(children)
np.random.seed(0)
coef_sph = np.random.randn(nbasis, nsph)

nsph = 1413
p = 6
ngrid = 110


In [5]:
t0 = time()
coef_ddpcm = mydx.ddpcm_dx(coef_sph)
print("ddPCM: ", time()-t0, " seconds")

ddPCM:  22.376887798309326  seconds


In [6]:
t0 = time()
coef_out = pcm_fmm.pcm_matvec_grid_treecode(csph, rsph, grid, w, vgrid, ui, p, vscales[:(p+1)*(p+1)], ind, \
                                   cluster, children, cnode, rnode, snode, coef_sph)
print("TREECODE: ", time()-t0, " seconds")
print(np.linalg.norm(coef_ddpcm-coef_out), np.linalg.norm(coef_ddpcm))

TREECODE:  11.47800898551941  seconds
0.05281673994514162 541.4104754512564


In [7]:
t0 = time()
coef_out2 = pcm_fmm.pcm_matvec_grid_treecode2(csph, rsph, grid, w, vgrid, ui, p, p, vscales, ind, \
                                   cluster, children, cnode, rnode, coef_sph)
print("TREECODE2: ", time()-t0, " seconds")
print(np.linalg.norm(coef_ddpcm-coef_out2), np.linalg.norm(coef_ddpcm))

TREECODE2:  11.630457878112793  seconds
0.16609022243495122 541.4104754512564


In [8]:
lwork = nsph*400
iwork = np.zeros(1, dtype=np.int32)
jwork = np.zeros(1, dtype=np.int32)
work = np.zeros((3,lwork), dtype=np.int32, order='F')
nnfar, nfar, nnnear, nnear = pcm_fmm.tree_get_farnear_work(children, cnode, rnode, iwork, jwork, work)
if(iwork[0] != jwork[0]+1):
    print("LWORK IS TOO SMALL")
else:
    sfar, far, snear, near = pcm_fmm.tree_get_farnear(jwork[0], work, nnfar, nfar, nnnear, nnear)
    #print(nnnear, nnear, snear, near)
    #print(nnfar, nfar, sfar, far)
    t0 = time()
    coef_out_fmm = pcm_fmm.pcm_matvec_grid_fmm(csph, rsph, grid, w, vgrid, ui, p, p, vscales, ind, cluster, \
                                               children, cnode, rnode, sfar, far, snear, near, coef_sph)
    print("FMM: ", time()-t0, " seconds")
    print(np.linalg.norm(coef_ddpcm-coef_out_fmm), np.linalg.norm(coef_ddpcm))
    #print(coef_out_fmm)
    #print(coef_out_fmm-coef_out)
    #print(coef_out)

FMM:  5.732115983963013  seconds
0.22924418540253869 541.4104754512564
